# Applying Advanced Transformations

In [57]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Importing the OS and JSON Modules
import os,json

In [58]:
# Load the files
names = pd.read_csv('Data/superhero_info - superhero_info.csv')
power = pd.read_csv('Data/superhero_powers - superhero_powers.csv')

## Clean the files and combine them into one final DataFrame.

In [59]:
# Display basic info for names dataframe
names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [60]:
# Display basic info for power dataframe
power.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [61]:
# Make all the columns lowercase
names.columns = names.columns.str.lower()
power.columns = power.columns.str.lower()

In [62]:
# Check existing format for the doubled column
names['hero|publisher'].head(5)

0            A-Bomb|Marvel Comics
1    Abe Sapien|Dark Horse Comics
2              Abin Sur|DC Comics
3       Abomination|Marvel Comics
4     Absorbing Man|Marvel Comics
Name: hero|publisher, dtype: object

In [63]:
# Seperate these pieces of information
names['hero|publisher'].str.split(' ',expand=True)

,0,1,2,3
0,A-Bomb|Marvel,Comics,None,None
1,Abe,Sapien|Dark,Horse,Comics
2,Abin,Sur|DC,Comics,None
3,Abomination|Marvel,Comics,None,None
4,Absorbing,Man|Marvel,Comics,None
...,...,...,...,...
458,Yellowjacket|Marvel,Comics,None,None
459,Yellowjacket,II|Marvel,Comics,None
460,Yoda|George,Lucas,None,None
461,Zatanna|DC,Comics,None,None


## Split hero and publisher

In [64]:
# split on '|' and save the 2 new columns for the dataframe
names[['hero','publisher']] = names['hero|publisher'].str.split('|',expand=True)

# Drop the original column 
names = names.drop(columns=['hero|publisher'])

# Display the new dataframe
names.head(2)

,gender,race,alignment,hair color,eye color,skin color,measurements,hero,publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


## Modify and Concatenate new columns

In [65]:
# Display the measurments type
m = names.loc[0,"measurements"]
print(type(m))
m

<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [66]:
# test the quotations replacement
m = m.replace("'",'"')
m

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [67]:
# Use .str.replace to replace all single quotes
names['measurements'] = names['measurements'].str.replace("'",'"')

# Apply the json.loads to the full column
names['measurements'] = names['measurements'].apply(json.loads)
names['measurements'].head()

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: measurements, dtype: object

In [68]:
# Use pd.Series to convert each key in the dictionary to new column
hw = names['measurements'].apply(pd.Series)
hw

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [69]:
# concatenate the new columns with the original dataframe
names = pd.concat((names, hw), axis = 1)
names.head(2)

,gender,race,alignment,hair color,eye color,skin color,measurements,hero,publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [70]:
# Drop the measurments column
names = names.drop(columns=['measurements'])

In [71]:
# Display new info
names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   gender      463 non-null    object
 1   race        463 non-null    object
 2   alignment   463 non-null    object
 3   hair color  463 non-null    object
 4   eye color   463 non-null    object
 5   skin color  463 non-null    object
 6   hero        463 non-null    object
 7   publisher   463 non-null    object
 8   Height      463 non-null    object
 9   Weight      463 non-null    object
dtypes: object(10)
memory usage: 36.3+ KB


In [72]:
# Display values for Height
names['Height'].value_counts()

183.0 cm    53
188.0 cm    44
180.0 cm    37
185.0 cm    34
178.0 cm    33
175.0 cm    32
168.0 cm    27
165.0 cm    26
170.0 cm    19
193.0 cm    18
191.0 cm    18
198.0 cm    18
173.0 cm    16
196.0 cm    10
201.0 cm    10
163.0 cm     8
203.0 cm     5
157.0 cm     5
213.0 cm     5
211.0 cm     4
244.0 cm     4
218.0 cm     3
155.0 cm     3
229.0 cm     3
137.0 cm     2
366.0 cm     2
206.0 cm     2
305.0 cm     2
122.0 cm     2
226.0 cm     2
279.0 cm     1
234.0 cm     1
15.2 cm      1
160.0 cm     1
64.0 cm      1
259.0 cm     1
287.0 cm     1
71.0 cm      1
701.0 cm     1
62.5 cm      1
876.0 cm     1
142.0 cm     1
975.0 cm     1
267.0 cm     1
257.0 cm     1
66.0 cm      1
Name: Height, dtype: int64

In [73]:
# Display values for weight
names['Weight'].value_counts()

81.0 kg     22
79.0 kg     21
54.0 kg     20
90.0 kg     19
86.0 kg     15
            ..
268.0 kg     1
16.0 kg      1
167.0 kg     1
45.0 kg      1
17.0 kg      1
Name: Weight, Length: 128, dtype: int64

In [74]:
# Create a new column without characters for height 
names['height (cm)'] = names['Height'].str.replace("cm",'')

In [75]:
# Create a new column without characters for weight
names['weight (kg)'] = names['Weight'].str.replace("kg",'')

In [76]:
# drop the original columns
names = names.drop(columns=['Height', 'Weight'])

In [77]:
# Change the 2 new columns to integers
names['height (cm)'] = names['height (cm)'].astype(float)

In [78]:
# Change the 2 new columns to integers
names['weight (kg)'] = names['weight (kg)'].astype(float)

#### After looking back at the lp, I realized I could have used the loop code listed in step 2 of the advanced transformations instead.

In [79]:
names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   gender       463 non-null    object 
 1   race         463 non-null    object 
 2   alignment    463 non-null    object 
 3   hair color   463 non-null    object 
 4   eye color    463 non-null    object 
 5   skin color   463 non-null    object 
 6   hero         463 non-null    object 
 7   publisher    463 non-null    object 
 8   height (cm)  463 non-null    float64
 9   weight (kg)  463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


In [80]:
names.head(5)

,gender,race,alignment,hair color,eye color,skin color,hero,publisher,height (cm),weight (kg)
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0


# Filter the powers data

In [81]:
# Display info 
power.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [82]:
# Check the value counts
power.value_counts('powers')

powers
Intelligence                                                                                                                                                                                                                                                                                                                                                            8
Durability,Super Strength                                                                                                                                                                                                                                                                                                                                               5
Electrokinesis                                                                                                                                                                                                                                                               

In [83]:
# Split on the comma because the LP told me to
power['powers'] = power['powers'].str.split(',')
power.head()

,hero_names,powers
0,3-D Man,"[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,[Lantern Power Ring]
4,Abomination,"[Accelerated Healing, Intelligence, Super Stre..."


## One Hot Explode

In [84]:
## exploding the column of lists
exploded = power.explode('powers')
exploded[['hero_names','powers']].head(5)

,hero_names,powers
0,3-D Man,Agility
0,3-D Man,Super Strength
0,3-D Man,Stamina
0,3-D Man,Super Speed
1,A-Bomb,Accelerated Healing


In [85]:
## saving the unique values from the exploded column
cols_to_make = exploded['powers'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [86]:
## Using a For Loop and .str.contains to create the new columns
for col in cols_to_make:
    power[col] = power['powers'].str.contains(col, regex= False)
power.head()

C:\Users\ridge\AppData\Local\Temp\ipykernel_1404\1956464206.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  power[col] = power['powers'].str.contains(col, regex= False)
C:\Users\ridge\AppData\Local\Temp\ipykernel_1404\1956464206.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  power[col] = power['powers'].str.contains(col, regex= False)
C:\Users\ridge\AppData\Local\Temp\ipykernel_1404\1956464206.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

,hero_names,powers,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,[Lantern Power Ring],False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [87]:
# Drop powers column
power.drop(columns=['powers'], inplace=True)
power.head()

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,True,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Combine the dataframes

In [88]:
# Merge the dataframes together
df = pd.merge(names, power, left_on='hero', right_on='hero_names')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 462
Columns: 178 entries, gender to Changing Armor
dtypes: bool(167), float64(2), object(9)
memory usage: 118.9+ KB


In [89]:
df.head()

,gender,race,alignment,hair color,eye color,skin color,hero,publisher,height (cm),weight (kg),...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,False,False,False,False,False,False,False,False,False,False


## Compare the average weight of super powers who have Super Speed to those who do not.

In [90]:
# Change weight to pounds instead of kilograms
df['weight (kg)'] = df['weight (kg)'].apply(lambda x: (x*2.20462262))

In [91]:
# Rename weight column
df.rename({'weight (kg)': 'weight (lbs)'}, axis=1, inplace=True)

In [92]:
# Average weight in pounds for heros who have and do not have super speed power
speed = df.groupby('Super Speed')['weight (lbs)'].mean()
speed.round(2)

Super Speed
False    224.37
True     285.29
Name: weight (lbs), dtype: float64

## What is the average height of heroes for each publisher?

In [93]:
# Change height to inches instead of centimeters
df['height (cm)'] = df['height (cm)'].apply(lambda x: (x/2.54))

In [94]:
# Convert inches to feet
df['height (cm)'] = df['height (cm)'].apply(lambda x: (x/12))

In [95]:
# Rename height column
df.rename({'height (cm)': 'height (ft)'}, axis=1, inplace=True)
df.head()

,gender,race,alignment,hair color,eye color,skin color,hero,publisher,height (ft),weight (lbs),...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,6.660105,972.238575,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,6.266404,143.300470,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,6.069554,198.416036,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,6.660105,972.238575,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,6.332021,268.963960,...,False,False,False,False,False,False,False,False,False,False


In [96]:
# Average height by publisher in feet
height = df.groupby('publisher')['height (ft)'].mean()
height.round(2)

publisher
DC Comics            5.97
Dark Horse Comics    5.80
George Lucas         5.24
Image Comics         6.92
Marvel Comics        6.28
Shueisha             5.63
Star Trek            5.95
Team Epic TV         5.93
Unknown              5.84
Name: height (ft), dtype: float64